In [1]:
from PIL import Image
from torchvision import datasets
import os
import shutil
import random

def crop_roof_area(image, height_ratio=0.3):
    width, height = image.size
    crop_height = int(height * height_ratio)
    return image.crop((0, 0, width, crop_height))

def generate_classwise_roof_crops(data_root, save_root, ratio=0.2, height_ratio=0.4):
    dataset = datasets.ImageFolder(data_root)
    class_to_idx = dataset.class_to_idx
    idx_to_class = {v: k for k, v in class_to_idx.items()}

    os.makedirs(save_root, exist_ok=True)

    classwise_paths = {cls: [] for cls in dataset.classes}
    for path, label in dataset.samples:
        classwise_paths[idx_to_class[label]].append(path)

    for cls_name, paths in classwise_paths.items():
        cls_out_dir = os.path.join(save_root, cls_name)
        os.makedirs(cls_out_dir, exist_ok=True)

        num_to_generate = int(len(paths) * ratio)
        selected_paths = random.sample(paths, num_to_generate)

        for i, img_path in enumerate(selected_paths):
            img = Image.open(img_path).convert("RGB")
            roof = crop_roof_area(img, height_ratio=height_ratio)
            new_name = f"roof_{i}_{os.path.basename(img_path)}"
            roof.save(os.path.join(cls_out_dir, new_name))

        # ✅ 원본도 함께 복사
        for path in paths:
            shutil.copy(path, os.path.join(cls_out_dir, os.path.basename(path)))

    print(f"✅ 클래스별 crop 비율 {ratio:.0%} 적용 완료. 저장 위치: {save_root}")


# 🔧 실행 파라미터 지정
data_root = "../data/train2"  # 원본 학습 데이터 폴더
save_root = "../data/train2_augmented"  # 저장할 새 폴더
crop_ratio = 0.2  # 각 클래스에서 crop 생성할 비율
roof_height_ratio = 0.4  # 이미지 상단 몇 % 자를지

# 🚀 함수 실행
generate_classwise_roof_crops(
    data_root=data_root,
    save_root=save_root,
    ratio=crop_ratio,
    height_ratio=roof_height_ratio
)